# Layer 1: Physical Map - Database Structure and Data Exploration

Interactive exploration of table sizes, row counts, column profiling, nullness analysis, and data distributions.

**Author:** Data Archaeologist Team  
**Version:** 2.0  
**Date:** 2025-08-28

In [7]:
# Essential imports for data analysis and visualization
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Data Archaeologist imports - corrected class names
sys.path.append('..')
from data_archaeologist.core.database_connection import DatabaseConnection
from data_archaeologist.layer1_physical.database_inventory import DatabaseInventory
from data_archaeologist.layer1_physical.table_sizing import TableSizingAnalyzer
from data_archaeologist.layer1_physical.column_profiling import ColumnProfiler

print("✓ All imports successful!")

✓ All imports successful!


In [8]:
!pip install matplotlib.pyplot

ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot


In [9]:
# Configuration and Environment Setup
import json

config_file = '../config.json'

with open(config_file, 'r') as f:
    config = json.load(f)

environments = list(config['environments'].keys())
analysis_settings = config.get('analysis_settings', {})

print(f"Available environments: {environments}")
print(f"Analysis settings: {analysis_settings}")

# Initialize database connection (staging environment)
db_conn = DatabaseConnection(config_file)
print("✓ Database connection initialized")

Available environments: ['staging', 'production', 'backup']
Analysis settings: {'enable_parallel_execution': True, 'max_workers': 3, 'connection_pool_size': 10, 'query_timeout': 300, 'large_table_threshold': 1000000, 'sample_size_for_analysis': 10000, 'top_n_tables': 20, 'sampling_rows': 100000, 'parallel_envs': True}
✓ Database connection initialized


In [13]:
# Interactive environment, schema, and table selection
import ipywidgets as widgets
from IPython.display import display, clear_output

print("✓ Setting up interactive interface...")

# Environment selector
env_dropdown = widgets.Dropdown(
    options=environments,
    value=environments[0] if environments else None,
    description='Environment:',
    style={'description_width': 'initial'}
)

print("✓ Environment dropdown created")

# For now, let's create a simplified version without the complex interactions
display(widgets.VBox([
    widgets.HTML("<h3>Data Archaeologist - Layer 1 Physical Map</h3>"),
    env_dropdown,
    widgets.HTML("<p>Selected Environment: Use the dropdown above to select environment</p>")
]))

print("✓ Interactive interface initialized")

✓ Setting up interactive interface...
✓ Environment dropdown created


✓ Interactive interface initialized


In [14]:
# Visualize top-N tables by size and rows

def analyze_top_tables(environment, n=20):
    """Analyze and visualize top N tables by size."""
    print(f"Analyzing top {n} tables in {environment}...")
    
    try:
        # Get table summary
        table_summary = get_table_summary(db_connection, environment)
        
        if not table_summary:
            print("No tables found")
            return
        
        # Convert to DataFrame
        df = pd.DataFrame(table_summary)
        df = df.head(n)  # Top N tables
        
        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Top Tables by Size (Bytes)',
                'Top Tables by Row Count',
                'Size vs Row Count Correlation',
                'Schema Distribution'
            ),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": False}, {"type": "pie"}]]
        )
        
        # Top tables by size
        fig.add_trace(
            go.Bar(
                x=df['full_table_name'],
                y=df['size_bytes'],
                name='Size (Bytes)',
                text=df['size_human'],
                textposition='auto'
            ),
            row=1, col=1
        )
        
        # Top tables by row count
        fig.add_trace(
            go.Bar(
                x=df['full_table_name'],
                y=df['rows'],
                name='Row Count',
                marker_color='orange'
            ),
            row=1, col=2
        )
        
        # Size vs Row correlation
        fig.add_trace(
            go.Scatter(
                x=df['rows'],
                y=df['size_bytes'],
                mode='markers',
                text=df['full_table_name'],
                name='Tables',
                marker=dict(size=8, color='green')
            ),
            row=2, col=1
        )
        
        # Schema distribution
        schema_counts = df['schema'].value_counts()
        fig.add_trace(
            go.Pie(
                labels=schema_counts.index,
                values=schema_counts.values,
                name='Schemas'
            ),
            row=2, col=2
        )
        
        # Update layout
        fig.update_layout(
            height=800,
            title_text=f"Physical Analysis - {environment.title()} Environment",
            showlegend=False
        )
        
        # Rotate x-axis labels for better readability
        fig.update_xaxes(tickangle=45, row=1, col=1)
        fig.update_xaxes(tickangle=45, row=1, col=2)
        
        fig.show()
        
        # Summary statistics
        print(f"\n📊 Summary Statistics:")
        print(f"Total tables analyzed: {len(df)}")
        print(f"Largest table: {df.iloc[0]['full_table_name']} ({df.iloc[0]['size_human']})")
        print(f"Most rows: {df.loc[df['rows'].idxmax()]['full_table_name']} ({df['rows'].max():,} rows)")
        print(f"Total size: {df['size_bytes'].sum() / (1024**3):.2f} GB")
        print(f"Total rows: {df['rows'].sum():,}")
        
        return df
        
    except Exception as e:
        print(f"❌ Error in analysis: {e}")
        return None

# Interactive analysis button
analyze_button = widgets.Button(
    description='Analyze Top Tables',
    button_style='primary',
    icon='chart-bar'
)

analysis_output = widgets.Output()

def on_analyze_click(b):
    with analysis_output:
        analysis_output.clear_output()
        env = env_dropdown.value
        if env:
            analyze_top_tables(env, top_n_tables)
        else:
            print("Please select an environment first")

analyze_button.on_click(on_analyze_click)

display(widgets.VBox([
    analyze_button,
    analysis_output
]))

In [15]:
# Column-level analysis for selected table

def analyze_table_columns(environment, schema, table):
    """Analyze columns of a specific table."""
    print(f"Analyzing columns of {schema}.{table}...")
    
    try:
        # Get column information
        column_query = """
        SELECT 
            column_name,
            data_type,
            character_maximum_length,
            is_nullable,
            column_default,
            ordinal_position
        FROM information_schema.columns
        WHERE table_schema = %s AND table_name = %s
        ORDER BY ordinal_position
        """
        
        columns = db_connection.execute_query(environment, column_query, (schema, table))
        
        if not columns:
            print("No column information found")
            return
        
        # Convert to DataFrame
        columns_df = pd.DataFrame(columns)
        
        # Analyze nullness
        null_analysis = []
        
        for col in columns:
            col_name = col['column_name']
            
            # Get null percentage
            null_query = f"""
            SELECT 
                COUNT(*) as total_rows,
                COUNT(\"{col_name}\") as non_null_rows,
                (COUNT(*) - COUNT(\"{col_name}\")) as null_rows,
                ROUND(((COUNT(*) - COUNT(\"{col_name}\")) * 100.0 / COUNT(*)), 2) as null_percentage
            FROM \"{schema}\".\"{table}\"
            """
            
            try:
                result = db_connection.execute_query(environment, null_query)
                if result:
                    null_analysis.append({
                        'column_name': col_name,
                        'data_type': col['data_type'],
                        'is_nullable': col['is_nullable'],
                        'null_percentage': result[0]['null_percentage'],
                        'null_rows': result[0]['null_rows'],
                        'total_rows': result[0]['total_rows']
                    })
            except Exception as e:
                print(f"Warning: Could not analyze nullness for {col_name}: {e}")
        
        # Create visualizations
        if null_analysis:
            null_df = pd.DataFrame(null_analysis)
            
            # Create subplots
            fig = make_subplots(
                rows=2, cols=2,
                subplot_titles=(
                    'Null Percentage by Column',
                    'Data Type Distribution',
                    'Nullable vs Non-Nullable Columns',
                    'Column Details'
                ),
                specs=[[{"secondary_y": False}, {"type": "pie"}],
                       [{"type": "pie"}, {"type": "table"}]]
            )
            
            # Null percentage chart
            fig.add_trace(
                go.Bar(
                    x=null_df['column_name'],
                    y=null_df['null_percentage'],
                    name='Null %',
                    marker_color='red'
                ),
                row=1, col=1
            )
            
            # Data type distribution
            type_counts = columns_df['data_type'].value_counts()
            fig.add_trace(
                go.Pie(
                    labels=type_counts.index,
                    values=type_counts.values,
                    name='Data Types'
                ),
                row=1, col=2
            )
            
            # Nullable distribution
            nullable_counts = columns_df['is_nullable'].value_counts()
            fig.add_trace(
                go.Pie(
                    labels=['Nullable' if x == 'YES' else 'Not Nullable' for x in nullable_counts.index],
                    values=nullable_counts.values,
                    name='Nullable'
                ),
                row=2, col=1
            )
            
            # Column details table
            fig.add_trace(
                go.Table(
                    header=dict(
                        values=['Column', 'Type', 'Nullable', 'Null %'],
                        fill_color='paleturquoise',
                        align='left'
                    ),
                    cells=dict(
                        values=[
                            null_df['column_name'],
                            null_df['data_type'],
                            null_df['is_nullable'],
                            null_df['null_percentage']
                        ],
                        fill_color='lavender',
                        align='left'
                    )
                ),
                row=2, col=2
            )
            
            fig.update_layout(
                height=800,
                title_text=f"Column Analysis - {schema}.{table}",
                showlegend=False
            )
            
            fig.update_xaxes(tickangle=45, row=1, col=1)
            
            fig.show()
            
            # Summary
            print(f"\n📊 Column Summary:")
            print(f"Total columns: {len(columns_df)}")
            print(f"Nullable columns: {len(columns_df[columns_df['is_nullable'] == 'YES'])}")
            print(f"Columns with nulls: {len(null_df[null_df['null_percentage'] > 0])}")
            print(f"Highest null percentage: {null_df['null_percentage'].max():.2f}%")
            
            return null_df
        
    except Exception as e:
        print(f"❌ Error in column analysis: {e}")
        return None

# Interactive column analysis button
column_button = widgets.Button(
    description='Analyze Selected Table Columns',
    button_style='info',
    icon='table'
)

column_output = widgets.Output()

def on_column_click(b):
    with column_output:
        column_output.clear_output()
        env = env_dropdown.value
        schema = schema_dropdown.value
        table = table_dropdown.value
        
        if env and schema and table:
            analyze_table_columns(env, schema, table)
        else:
            print("Please select environment, schema, and table first")

column_button.on_click(on_column_click)

display(widgets.VBox([
    column_button,
    column_output
]))

## Layer 1 Analysis Complete ✅

This notebook provided physical-level analysis including:

- **Table Size Analysis** - Top N tables by size and row count
- **Schema Distribution** - How tables are distributed across schemas
- **Column Profiling** - Data types, nullness analysis, and column characteristics
- **Data Quality Insights** - Null percentages and data integrity overview

### Key Insights:
- Use the interactive widgets to explore different environments and tables
- Focus on tables with high null percentages for data quality improvements
- Large tables may benefit from partitioning or archival strategies
- Schema distribution can inform database organization decisions

### Next Steps:
- **02_layer2_logical_blueprint.ipynb** - Explore relationships, keys, and logical structure
- **03_layer3_business_story.ipynb** - Discover business processes and insights
- **04_multi_env_parallel_run.ipynb** - Compare findings across environments